### 导入模拟数据

使用模拟数据，测试模型

In [1]:
import os
import numpy as np
os.chdir("/home/num_24/桌面/mywork/github/TFDeepSurv")
os.getcwd()

np.random.seed(10)

In [2]:
from dataset import SimulatedData

data_config = SimulatedData(10000, num_features = 9)
data = data_config.generate_data(2000)

In [3]:
N = 10
print(data['x'][0:N])
print(data['e'][0:N])
print(data['t'][0:N])

[[ 0.54264128 -0.95849609  0.26729646  0.49760777 -0.00298598 -0.55040669
  -0.60387427  0.52106142 -0.66177833]
 [-0.82332039  0.37071964  0.90678668 -0.99210346  0.02438453  0.62524194
   0.22505213  0.44351062 -0.41624787]
 [ 0.83554822  0.42915156  0.08508874 -0.71565992 -0.25331849  0.34826723
  -0.11633365 -0.13197201  0.23553395]
 [ 0.02627649  0.30079436  0.20207791  0.61044639  0.0432943   0.81729776
  -0.36152783 -0.81908131 -0.39859989]
 [-0.77203125  0.65736264 -0.90620738  0.25257429  0.09517231  0.638574
  -0.6021049   0.71370059 -0.29669473]
 [ 0.5092954  -0.40807658  0.76787293 -0.34897673 -0.66996819 -0.21494152
  -0.81307924  0.64221132 -0.69769597]
 [-0.2317711   0.88852143  0.97525096 -0.08739091  0.6522457  -0.49725172
   0.19474329  0.80566353  0.0691159 ]
 [ 0.18040273 -0.92143649 -0.28563648 -0.84077382 -0.38908017 -0.33856139
   0.54766059 -0.92008156 -0.14101565]
 [-0.37014624  0.2729823  -0.30730569 -0.91380531  0.75983036  0.52648115
   0.75619328 -0.1649817

In [4]:
sum(data['e']) 

1356

In [5]:
import LDeepSurv

Using TensorFlow backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [6]:
n_features = data['x'].shape[1]
print("features: ", n_features)
model = LDeepSurv.LDeepSurv((None, n_features), hidden_layer_sizes = [10, 5])

features:  9


ValueError: length not known: e [id A]
